In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import datetime 
%matplotlib inline

train_df = pd.read_csv("train.csv", parse_dates=['timestamp'])
test_df = pd.read_csv("test.csv")

Quantify categorical columns

In [ ]:
string_dict = {'no data':-1, 
              'poor':0,
              'satisfactory':1,
              'good': 2,
              'excellent':3,
              'yes': 1,
              'no': 0,
              'Investment': 1,
              'OwnerOccupier': 0}
for key in [key for key in dict(train_df.dtypes) if dict(train_df.dtypes)[key] in ['O'] and key != 'sub_area']:
    train_df[key] = train_df[key].map(string_dict)

Add one column for each sub_area

In [ ]:
for area in train_df['sub_area'].unique():
    train_df.insert(train_df.shape[1],'sub_area_'+area, 0)
    train_df['sub_area_'+ area] = (train_df['sub_area'] == area).astype(int)
    
train_df = train_df.drop('sub_area', 1)

Edit 'build_year'

In [ ]:
train_df.ix[train_df['build_year']==0,'build_year'] = None

train_df['build_year'] = train_df['build_year'].apply(lambda x: 2017-x)

Edit'timestamp' as days to date(2016,6,1)

In [ ]:
train_df['timestamp'] = train_df['timestamp'].apply(lambda x: (datetime.datetime(2016,6,1)-x).days)

Normalization

In [ ]:
train_df_dscb = train_df.describe(percentiles=[.96])

In [ ]:
for col in [x for x in train_df_dscb.columns.values if x not in ['id', 'price_doc'] and x[0:9] != 'sub_area_']:
    print(col)
    train_df[col] = train_df.ix[:, col].apply(lambda x: (x-train_df_dscb[col]['min'])/(train_df_dscb[col]['96%']-train_df_dscb[col]['min']))

Fill the null cells with -1

In [ ]:
train_df = train_df.fillna(-1)

In [ ]:
train_df.describe(percentiles=[.96])

In [ ]:
train_df_dscb